In [53]:
import pandas as pd
import datetime as dt

In [54]:
# Load in the marriage and divorce files
marriages = pd.read_csv("../data/Base-Matrimonios-2000-al-31_03_2015seg.csv")
divorces = pd.read_csv("../data/bases-divorcios-2000-31_03_2015-csv.csv")

In [55]:
##### Creating marriage df with only marriage date, and both DOB's #####
# Isolate those columns
marriages = marriages[["Fecha","ElFecNac","EllaFecNac"]]
# Rename the columns to translate to english
marriages = marriages.rename(columns={"Fecha":"Marriage_Date","ElFecNac":"DOB1","EllaFecNac":"DOB2"})
# Split the datetime columns into date and time columns
marriages[["Marriage_Date","Marriage_Time"]] = marriages["Marriage_Date"].str.split("T",expand=True)
marriages[["DOB1","DOB1_Time"]] = marriages["DOB1"].str.split("T",expand=True)
marriages[["DOB2","DOB2_Time"]] = marriages["DOB2"].str.split("T",expand=True)
# Drop the time columns
marriages = marriages[["Marriage_Date","DOB1","DOB2"]]
marriages.head()

,Marriage_Date,DOB1,DOB2
0,2000-01-03,1974-08-20,1980-01-01
1,2000-01-05,1980-02-12,1981-12-12
2,2000-01-05,1978-11-26,1979-09-04
3,2000-01-06,1974-10-30,1978-12-04
4,2000-01-06,1937-03-13,1956-09-11


In [56]:
#### Creating divorce df with only marriage date, both DOB's, and a flag for divorce ####
# Isolate those columns
divorces = divorces[["Fecha","F_Naci_Do","F_Naci_Da"]]
# Rename the columns to translate to english
divorces = divorces.rename(columns={"Fecha":"Marriage_Date","F_Naci_Do":"DOB1","F_Naci_Da":"DOB2"})
# Drop nulls - can't merged if missing
divorces = divorces.dropna()
# Reformt date columns to be able to merge on marriage data
divorces['Marriage_Date'] = pd.to_datetime(divorces['Marriage_Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
divorces['DOB1'] = pd.to_datetime(divorces['DOB1'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
divorces['DOB2'] = pd.to_datetime(divorces['DOB2'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
# Add flag for divorce
divorces["divorce"] = 1
divorces.head()

,Marriage_Date,DOB1,DOB2,divorce
0,2000-06-26,1975-12-18,1983-01-08,1
2,1975-12-18,1955-02-22,1947-03-21,1
4,1998-11-14,1975-10-30,1978-10-13,1
5,1995-01-20,1973-03-28,1976-06-14,1
6,1991-08-16,1970-12-13,1971-11-04,1


In [57]:
# Merge on Marriage Date, DOB2
df = pd.merge(marriages, divorces, how='left', on=["Marriage_Date","DOB1","DOB2"])
# Fill the null divorce flag -- these are marriages that have not ended in divorce
df["divorce"] = df["divorce"].fillna(0)

In [58]:
# Show the number of divorces and marriages
df["divorce"].value_counts()

0.0    23465
1.0     1284
Name: divorce, dtype: int64

In [59]:
# Isolate divorces
divorces_df = df.loc[df["divorce"]==1,:]
# Isolate marriages
marriages_df = df.loc[df["divorce"]==0,:]
# Take a random sample of the successful marriages equal in number to number of divorces
marriages_sample = marriages_df.sample(n=1284, random_state=23)
# Concat these two to create dataframe with equal number of successful and unsuccesful marriages
marriage_success = pd.concat([marriages_sample, divorces_df])

In [60]:
marriage_success

,Marriage_Date,DOB1,DOB2,divorce
22554,2014-02-19,1983-11-17,1990-03-25,0.0
12203,2008-02-15,1987-01-26,1987-06-09,0.0
17722,2011-06-11,1981-05-05,1982-09-02,0.0
7319,2005-01-25,1985-10-24,1977-06-10,0.0
21310,2013-05-14,1993-04-29,1988-06-09,0.0
...,...,...,...,...
21332,2013-05-17,1977-12-29,1990-02-07,1.0
21359,2013-05-25,1988-04-14,1988-08-19,1.0
21837,2013-10-19,1981-08-21,1983-09-29,1.0
22478,2014-02-14,1985-02-01,1989-03-15,1.0


In [61]:
# Export to csv
marriage_success.to_csv("../data/marriage_success.csv", header=True)